In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
from PIL import Image
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_path = '/kaggle/input/butterfly-image-classification' 
train_csv_path = os.path.join(dataset_path, 'Training_set.csv')
test_csv_path = os.path.join(dataset_path, 'Testing_set.csv')
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

In [ ]:
# Step 3: Load the training CSV and explore the data
train_df = pd.read_csv("/kaggle/input/butterfly-image-classification/Training_set.csv")
print("Training set shape:", train_df.shape)
print("\nFirst few rows:")
print(train_df.head())
print("\nUnique classes:", train_df['label'].nunique())
print("Class distribution:")
print(train_df['label'].value_counts().head())

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y='label', data=train_df, order=train_df['label'].value_counts().index[:10])
plt.title('Top 10 Butterfly Classes Distribution')
plt.show()

In [ ]:
# Step 4: Encode labels
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")

In [ ]:
# Step 5: Define image loading and preprocessing function
IMG_SIZE = (224, 224)  # Standard size for many CNNs; adjust if needed for memory
def load_and_preprocess_image(image_path, label):
    """
    Load image, resize, normalize, and return as numpy array.
    """
    try:
        image = cv2.imread(image_path)
        if image is None:
            return None
        image = cv2.resize(image, IMG_SIZE)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = image.astype('float32') / 255.0  # Normalize to [0,1]
        return image, label
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        return None

In [ ]:
# Step 6: Load training images with preprocessing
# This may take time (~1000+ images); run on GPU for speed
print("Loading and preprocessing training images...")
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image_filename = row['filename']
    image_path = os.path.join(train_dir, image_filename)
    result = load_and_preprocess_image(image_path, row['label_encoded'])
    if result is not None:
        img, lbl = result
        train_images.append(img)
        train_labels.append(lbl)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

print(f"Loaded {len(train_images)} training images with shape {train_images.shape}")

In [ ]:
# Step 7: One-hot encode labels
train_labels_cat = to_categorical(train_labels, num_classes=num_classes)

In [ ]:
# Step 8: Split into train and validation sets (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_labels_cat, test_size=0.2, random_state=42, stratify=train_labels
)
print(f"Training set: {X_train.shape[0]} images")
print(f"Validation set: {X_val.shape[0]} images")

# Visualize a few sample images
plt.figure(figsize=(12, 8))
for i in range(6):
    plt.subplot(2, 3, i+1)
    idx = np.random.randint(0, X_train.shape[0])
    plt.imshow(X_train[idx])
    true_label = np.argmax(y_train[idx])
    plt.title(f"Class: {label_encoder.inverse_transform([true_label])[0]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Step 9: Data Augmentation (to prevent overfitting)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen.fit(X_train)

In [ ]:
# Step 10: Build the CNN Model
model = Sequential([
    # First Conv Block
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Second Conv Block
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Third Conv Block
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Fourth Conv Block
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Flatten and Dense Layers
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Step 11: Train the model
# Use GPU/TPU in Kaggle for faster training
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=20,  # Adjust based on time; start with 10-20
    validation_data=(X_val, y_val),
    verbose=1
)

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Step 12: Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# Step 13: Predict on test set
test_df = pd.read_csv(test_csv_path)
print("Test set shape:", test_df.shape)

test_images = []
test_filenames = test_df['filename'].values  # Assuming column name is 'filename'

print("Loading and preprocessing test images...")
for filename in test_filenames:
    image_path = os.path.join(test_dir, filename)
    result = load_and_preprocess_image(image_path, None)  # No label for test
    if result is not None:
        img, _ = result
        test_images.append(img)

test_images = np.array(test_images)
print(f"Loaded {len(test_images)} test images")

# Make predictions
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

In [ ]:
# Step 14: Create submission CSV
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'label': predicted_labels
})
submission_df.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created! Download and submit it to the competition.")

# Optional: Visualize some test predictions
plt.figure(figsize=(12, 8))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Predicted: {predicted_labels[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()